`Дисциплина: Методы и технологии машинного обучения`   
`Уровень подготовки: бакалавриат`   
`Направление подготовки: 01.03.02 Прикладная математика и информатика`   
`Семестр: осень 2021/2022`

In [1]:
# настройка ширины страницы блокнота .......................................
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))
# расширение watermark для вывода информации о версиях пакетов
# https://github.com/rasbt/watermark
%load_ext watermark

# Лабораторная работа №6: Машины опорных векторов
В практических примерах ниже показано:   

* как классифицировать данные с помощью модели SVM;  
* как использовать конвейеры для подгонки модели и применения её к новым данным;  

Точность всех моделей оценивается методом перекрёстной проверки по 5 блокам.

Данные: `winequality-white_for_lab` (источник: <https://github.com/aksyuk/MTML/blob/main/Labs/data/winequality-white_for_lab.csv>)

Список столбцов:

* `fixed_acidity` – постоянная кислотность;

* `volatile_acidity` – переменная кислотность;

* `citric_acid` – содержание лимонной кислоты;

* `residual_sugar` – остаточный сахар;

* `chlorides` – содержание хлоридов;

* `free_sulfur_dioxide` – содержание диоксида серы в свободном виде;

* `total_sulfur_dioxide` – общее содержание диоксида серы;
density – плотность;

* `pH` – кислотность;

* `sulphates` – содержание сульфатов;

* `alcohol` – содержание алкоголя;

* `quality` – балльная оценка качества вина, от 0 до 10;

* `Y` – целевая переменная: 1 = высокое качество (quality > 5), 0 = низкое (quality <= 5).

# Указания к выполнению

## Загружаем пакеты

In [2]:
# загрузка пакетов: инструменты --------------------------------------------
# работа с массивами
import numpy as np
# фреймы данных
import pandas as pd
# графики
import matplotlib as mpl
# стили и шаблоны графиков на основе matplotlib
import seaborn as sns
# перекодировка символьных показателей
from sklearn.preprocessing import LabelEncoder
# для таймера
import time
# загрузка пакетов: модели -------------------------------------------------
# SVM
from sklearn.svm import SVC
# логистическая рагрессия
from sklearn.linear_model import LogisticRegression
# стандартизация
from sklearn.preprocessing import StandardScaler
# метод главных компонент
from sklearn.decomposition import PCA
# конвейеры
from sklearn.pipeline import make_pipeline
# перекрёстная проверка и метод проверочной выборки
from sklearn.model_selection import cross_val_score, train_test_split
# для перекрёстной проверки и сеточного поиска
from sklearn.model_selection import KFold, GridSearchCV
# сводка по точности классификации
from sklearn.metrics import classification_report

In [3]:
# константы
# ядро для генератора случайных чисел
my_seed = 8
# создаём псевдоним для короткого обращения к графикам
plt = mpl.pyplot
# настройка стиля и отображения графиков
# примеры стилей и шаблонов графиков: 
# http://tonysyu.github.io/raw_content/matplotlib-style-gallery/gallery.html
mpl.style.use('seaborn-whitegrid')
sns.set_palette("Set2")
# раскомментируйте следующую строку, чтобы посмотреть палитру
# sns.color_palette("Set2")

## Загружаем данные

Набор данных можно загрузить напрямую по ссылке: <https://github.com/aksyuk/MTML/blob/main/Labs/data/winequality-white_for_lab.csv>

Справочник к данным доступен по адресу: <https://github.com/aksyuk/MTML/blob/main/Labs/data/CodeBook_winequality-white_for_lab.md>

Загружаем данные во фрейм и выясняем их размерность. Смотрим первые строки таблицы.

In [4]:
# загружаем данные
DF_raw = pd.read_csv('https://raw.githubusercontent.com/aksyuk/MTML/main/Labs/data/winequality-white_for_lab.csv')

# выясняем размерность фрейма
print('Число строк и столбцов в наборе данных:\n', DF_raw.shape)     

Число строк и столбцов в наборе данных:
 (4898, 13)


In [5]:
# первые строки
DF_raw.head(5)

,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulphates,alcohol,quality,Y
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6,1
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6,1
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6,1
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6,1
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6,1


In [6]:
# типы столбцов
DF_raw.dtypes

fixed_acidity           float64
volatile_acidity        float64
citric_acid             float64
residual_sugar          float64
chlorides               float64
free_sulfur_dioxide     float64
total_sulfur_dioxide    float64
density                 float64
pH                      float64
sulphates               float64
alcohol                 float64
quality                   int64
Y                         int64
dtype: object

In [7]:
DF_raw.isna().sum()

fixed_acidity           0
volatile_acidity        0
citric_acid             0
residual_sugar          0
chlorides               0
free_sulfur_dioxide     0
total_sulfur_dioxide    0
density                 0
pH                      0
sulphates               0
alcohol                 0
quality                 0
Y                       0
dtype: int64

In [8]:
DF = DF_raw.sample(frac = 0.85, random_state = my_seed)
DF_predict = DF_raw.drop(DF.index)

In [9]:
DF.describe()

,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulphates,alcohol,quality,Y
count,4163.000000,4163.000000,4163.000000,4163.000000,4163.000000,4163.000000,4163.000000,4163.000000,4163.000000,4163.000000,4163.000000,4163.000000,4163.000000
mean,6.853147,0.278675,0.333666,6.393935,0.046029,35.277925,138.093682,0.994018,3.187821,0.489181,10.519741,5.879414,0.664425
std,0.839406,0.101242,0.121310,5.115534,0.022604,16.664716,42.015077,0.003026,0.150405,0.114023,1.237346,0.885500,0.472248
min,3.800000,0.080000,0.000000,0.600000,0.009000,2.000000,9.000000,0.987110,2.720000,0.220000,8.000000,3.000000,0.000000
25%,6.300000,0.210000,0.270000,1.700000,0.036000,23.000000,108.000000,0.991700,3.090000,0.410000,9.500000,5.000000,0.000000
50%,6.800000,0.260000,0.320000,5.100000,0.043000,34.000000,134.000000,0.993700,3.180000,0.470000,10.400000,6.000000,1.000000
75%,7.300000,0.320000,0.390000,9.900000,0.050000,46.000000,167.000000,0.996135,3.280000,0.550000,11.400000,6.000000,1.000000
max,14.200000,1.100000,1.660000,65.800000,0.346000,146.500000,344.000000,1.038980,3.800000,1.080000,14.200000,9.000000,1.000000


## Преобразование исходных данных и построение моделей

В качестве альтернативных моделей рассмотрим SVM с различными вариантами ядер и логистическую регрессию. Причём предварительно преобразуем пространство исходных показателей с помощью метода главных компонент.

## Стандартизация и переход к главным компонентам

In [10]:
# стандартизация
sc = StandardScaler()
X_train_std = sc.fit_transform(DF.iloc[:, 0:12].values)

# оцениваем объяснённую главными компонентами дисперсию
pca = PCA()
X_train_pca = pca.fit_transform(X_train_std)

# считаем доли объяснённой дисперсии
frac_var_expl = pca.explained_variance_ratio_
print('Доли объяснённой дисперсии по компонентам в PLS:\n',
    np.around(frac_var_expl, 3),
    '\nОбщая сумма долей:', np.around(sum(frac_var_expl), 3))

Доли объяснённой дисперсии по компонентам в PLS:
 [0.28  0.132 0.115 0.09  0.082 0.078 0.064 0.059 0.046 0.029 0.024 0.002] 
Общая сумма долей: 1.0


Таким образом, первые две главные компоненты объясняют 41.2% разброса 12 объясняющих переменных.
Теперь объединим функции-преобразователи и оценщики в конвейер с помощью `Pipeline` и оценим точность логистической регрессии с помощью перекрёстной проверки.

## Модель логистической регрессии с перекрёстной проверкой

In [11]:
# данные для обучения моделей
X_train = DF.iloc[:, 0:12] 
y_train = DF.iloc[:, -1]

# объединяем в конвейер шкалирование, ГК с 2 компонентами и логит
pipe_lr = make_pipeline(StandardScaler(),
                        PCA(n_components = 2),
                        LogisticRegression(random_state = my_seed, 
                                           solver = 'lbfgs'))

# будем сохранять точность моделей в один массив
score = list()
score_models = list()

# считаем точность с перекрёстной проверкой, показатель Acc
cv = cross_val_score(estimator = pipe_lr, X = X_train, y = y_train, 
                     cv = 5, scoring='accuracy')

# записываем точность
score.append(np.around(np.mean(cv), 3)) 
score_models.append('sc_pca_logit')
print('Acc с перекрёстной проверкой',
      '\nдля модели', score_models[0], ':', score[0])

Acc с перекрёстной проверкой 
для модели sc_pca_logit : 0.685


In [12]:
X_train

,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulphates,alcohol,quality
4245,7.6,0.350,0.47,13.30,0.037,42.0,116.0,0.99822,3.04,0.50,9.2,5
945,6.7,0.150,0.29,5.00,0.058,28.0,105.0,0.99460,3.52,0.44,10.2,7
394,6.8,0.370,0.51,11.80,0.044,62.0,163.0,0.99760,3.19,0.44,8.8,5
3464,6.9,0.290,0.41,7.80,0.046,52.0,171.0,0.99537,3.12,0.51,9.6,5
4261,6.0,0.310,0.27,2.30,0.042,19.0,120.0,0.98952,3.32,0.41,12.7,7
...,...,...,...,...,...,...,...,...,...,...,...,...
2505,7.4,0.400,0.41,14.10,0.053,37.0,194.0,0.99886,3.20,0.63,9.4,6
1245,8.0,0.660,0.72,17.55,0.042,62.0,233.0,0.99990,2.92,0.68,9.4,4
4080,5.7,0.250,0.27,10.80,0.050,58.0,116.0,0.99592,3.10,0.50,9.8,6
558,5.9,0.210,0.24,12.10,0.044,53.0,165.0,0.99690,3.25,0.39,9.5,5


In [13]:
y_train

4245    0
945     1
394     0
3464    0
4261    1
       ..
2505    1
1245    0
4080    1
558     0
594     0
Name: Y, Length: 4163, dtype: int64

Таким образом, мы убедились в том, что X_train и y_train правильно определены.

## SVM с перекрёстной проверкой

Построим несколько вариантов модели SVM с различными ядерными функциями.

In [14]:
pipe_svc = make_pipeline(StandardScaler(), 
                         SVC(random_state = my_seed))

# настроим параметры SVM с помощью сеточного поиска
param_range = [0.0001, 0.001, 0.01, 0.1] 
param_grid = [{'svc__C': param_range,
               'svc__kernel': ['linear']},
              {'svc__C': param_range,
               'svc__gamma': param_range,
               'svc__kernel': ['rbf']},
              {'svc__C': param_range,
               'svc__gamma': param_range,
               'svc__degree' : [2, 3],
               'svc__kernel': ['poly']}]

# разбиения для перекрёстной проверки
kfold = KFold(n_splits = 5, random_state = my_seed, shuffle = True)
gs = GridSearchCV(estimator = pipe_svc, param_grid = param_grid, 
                  scoring = 'accuracy', refit = True, cv = kfold, 
                  n_jobs = -1)

# таймер
tic = time.perf_counter()

# запускаем сеточный поиск
gs = gs.fit(X_train, y_train)

# таймер
toc = time.perf_counter()
print(f"Сеточный поиск занял {toc - tic:0.2f} секунд", sep='')

Сеточный поиск занял 81.15 секунд


In [15]:
# точность лучшей модели
np.around(gs.best_score_, 3)

1.0

In [16]:
# параметры лучшей модели
# * ядерная функция
gs.best_estimator_.get_params()['svc__kernel']

'linear'

In [17]:
# * параметр регуляризации
gs.best_estimator_.get_params()['svc__C']

0.01

In [18]:
# * коэффициент ядерной функции (для ядер 'rbf', 'poly' и 'sigmoid')
gs.best_estimator_.get_params()['svc__gamma']

'scale'

In [19]:
# * степень полинома (для ядра 'poly')
gs.best_estimator_.get_params()['svc__degree']

3

📚 Подробности сеточного поиска

Посмотреть результаты сеточного поиска можно в объектах:
* gs.cv_results_['params'] – список сочетаний параметров;
* gs.cv_results_['mean_test_score'] – значения для сочетаний параметров
(средние по блокам перекрёстной проверки).


In [20]:
# записываем точность
score.append(np.around(gs.best_score_, 3))
score_models.append('sc_pca_svc')
print('Acc с перекрёстной проверкой',
 '\nдля модели', score_models[1], ':', score[1])

Acc с перекрёстной проверкой 
для модели sc_pca_svc : 1.0


# Метод kNN

Реализуем метод k-ближайших соседей с преобразованием PCA.

In [21]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline

In [22]:
k_range = list(range(1,32))
weight_options = ["uniform", "distance"]

pipe_knn = Pipeline(steps=[('standard', StandardScaler()), ('PCA', PCA(n_components = 5)), ('knn', KNeighborsClassifier())])
param_grid = {
    'knn__n_neighbors': list(range(1,32))
}

knn = KNeighborsClassifier()
kfold = KFold(n_splits = 5, random_state = my_seed, shuffle = True)
grid = GridSearchCV(pipe_knn, param_grid, cv=kfold, scoring='accuracy')

tic = time.perf_counter()
grid.fit(X_train,y_train)
toc = time.perf_counter()
print(f"Сеточный поиск занял {toc - tic:0.2f} секунд", sep='')

Сеточный поиск занял 12.11 секунд


Для повышения точности модели в PCA менялся параметр n_components, что позволило увеличить точность модели с 0,729 при n=1 до 0,895 при n=5.

In [23]:
score.append(np.around(grid.best_score_,3))
score_models.append('sc_pca_knn')

print('Acc с перекрёстной проверкой','\nдля модели',score_models[1],':',score[1])

Acc с перекрёстной проверкой 
для модели sc_pca_svc : 1.0


## Прогноз на отложенные наблюдения по лучшей модели

In [24]:
# сводка по точности моделей
pd.DataFrame({'Модель' : score_models, 'Acc' : score})

,Модель,Acc
0,sc_pca_logit,0.685
1,sc_pca_svc,1.000
2,sc_pca_knn,0.895


В данном случае модель PCA_SVC показывает большую точность, чем модель логистической регрессии.

Сделаем прогноз на отложенные наблюдения с помощью второго ансамбля.

In [25]:
# прогноз с помощью лучшей модели ансамбля с SVC
y_hat=gs.best_estimator_.predict(X=DF_predict.iloc[:, 0:12])
# точность# характеристики точности
print(classification_report(DF_predict.iloc[:,-1],y_hat))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       243
           1       1.00      1.00      1.00       492

    accuracy                           1.00       735
   macro avg       1.00      1.00      1.00       735
weighted avg       1.00      1.00      1.00       735

